<a href="https://colab.research.google.com/github/smfelixchoi/JEJU-contest/blob/master/Modeling_super_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression,Ridge,Lasso

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
df1 = pd.read_csv('/content/gdrive/My Drive/Jeju_credit contest/jeju_data_ver1/201901-202003.csv', header=0)
df2 = pd.read_csv('/content/gdrive/My Drive/Jeju_credit contest/jeju_data_ver1/202004.csv', header=0)

df = pd.concat([df1, df2])

### Data Preprocessing

In [5]:
df.rename(columns = {'REG_YYMM' : 'DATE', 'SEX_CTGO_CD':'SEX'}, inplace = True)
drop_columns = ['CARD_CCG_NM', 'HOM_SIDO_NM', 'HOM_CCG_NM']
df.drop(drop_columns, axis=1, inplace = True)

In [6]:
df = df.groupby(['DATE', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX', 'FLC'])['CSTMR_CNT','AMT','CNT'].sum().reset_index()
df['revisit']=np.round(1-df['CSTMR_CNT']/df['CNT'],5)

In [7]:
# revisit을 카테고리화하기
def get_revisit(data):
  if data>0:
    data='positive'
  elif data<0:
    data='negative'
  else:
    data='balanced'
  return data

In [8]:
df['revisit']=df['revisit'].apply(lambda x: get_revisit(x))

In [9]:
df.drop(['CSTMR_CNT','CNT'], axis = 1, inplace = True)
df.head()

,DATE,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX,FLC,AMT,revisit
0,201901,강원,건강보조식품 소매업,20s,1,1,311200,balanced
1,201901,강원,건강보조식품 소매업,30s,1,2,5169295,positive
2,201901,강원,건강보조식품 소매업,30s,2,2,8618550,positive
3,201901,강원,건강보조식품 소매업,40s,1,2,454420,balanced
4,201901,강원,건강보조식품 소매업,40s,1,3,6441165,positive


In [10]:
def get_year(df):
  df=str(df)
  return int(df[:4])

def get_month(df):
  df=str(df)
  return int(df[4:])

In [11]:
df['year']=df['DATE'].apply(lambda x:get_year(x))
df['month']=df['DATE'].apply(lambda x:get_month(x))

In [12]:
df.drop(['DATE'],axis=1,inplace=True)
df.head(1)

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX,FLC,AMT,revisit,year,month
0,강원,건강보조식품 소매업,20s,1,1,311200,balanced,2019,1


In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
# LabelEncoder 
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object': # object type column label-encoding/ 숫자는 알아서 분류
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder

df_org = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_org[column] = encoder.transform(df[column])

In [15]:
df_org.head(1)

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX,FLC,AMT,revisit,year,month
0,0,0,1,1,1,311200,0,2019,1


In [16]:
encoders

{'AGE': LabelEncoder(),
 'CARD_SIDO_NM': LabelEncoder(),
 'STD_CLSS_NM': LabelEncoder(),
 'revisit': LabelEncoder()}

### Modeling

In [17]:
def get_traintest(df):
  target=np.log1p(df['AMT'])
  X=df.drop(['AMT'],axis=1)
  X_train,X_test,y_train,y_test=train_test_split(X,target,test_size=0.2,random_state=0)
  return target, X, X_train,X_test,y_train,y_test

In [18]:
def get_best_params(model,params):
  grid_model=GridSearchCV(model,param_grid=params,scoring='neg_mean_squared_error',cv=5)
  grid_model.fit(X_train,y_train)
  rmsle=np.sqrt(-1*grid_model.best_score_)
  print('{0} 5 CV 평균 RMSLE: {1}, 최적 alpha: {2}'.format(model.__class__.__name__,np.round(rmsle,4),grid_model.best_params_))

In [19]:
def get_model_cv_prediction(model,X,target):
  neg_msle_scores=cross_val_score(model,X,target,scoring='neg_mean_squared_error',cv=5)
  rmsle_scores=np.sqrt(-1*neg_msle_scores)
  avg_rmsle=np.mean(rmsle_scores)
  print('###',model.__class__.__name__,'###')
  print('교차검증 개별 RMSLE:',rmsle_scores)
  print('교차검증 평균 RMSLE:{0:.4f} '.format(avg_rmsle))

In [20]:
def rmsle(model,y_test):
  pred=model.predict(X_test)
  mse=mean_squared_error(y_test,pred)
  rmse=np.sqrt(mse)
  return rmse

def get_rmsles(models):
  rmsles=[]
  for model in models:
    rmsle=rmsle(model,y_test)
    rmsles.append(rmsle)
  return rmsles

In [21]:
target,X,X_train,X_test,y_train,y_test=get_traintest(df_org)

In [22]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

In [23]:
#lgbm_params={'n_estimators':[1000,1200],'learning_rate':[0.5,0.8],'max_depth':[10,12]}
#lgbm_reg=LGBMRegressor(n_estimators=1000,learning_rate=0.5,max_depth=10,num_leaves=10)
#lgbm_reg.fit(X_train,y_train)
#rmsle(lgbm_reg,y_test)
#get_best_params(lgbm_reg,lgbm_params)

In [24]:
lgbm_reg=LGBMRegressor(n_estimators=2000,learning_rate=0.8,max_depth=10,num_leaves=15, n_jobs = -1)
lgbm_reg.fit(X_train,y_train)
rmsle(lgbm_reg,y_test)

0.4325254952921611

In [25]:
CARD_SIDO_NMs = df_org['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_org['STD_CLSS_NM'].unique()
AGEs          = df_org['AGE'].unique()
SEXs  = df_org['SEX'].unique()
FLCs          = df_org['FLC'].unique()
years         = [2020]
months        = [4, 7]
revisits=df_org['revisit'].unique()

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
      for AGE in AGEs:
        for SEX in SEXs:
          for FLC in FLCs:
            for year in years:
              for month in months:
                for revisit in revisits:
                  temp.append([CARD_SIDO_NM, STD_CLSS_NM, AGE, SEX, FLC, year, month,revisit])

temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX', 'FLC', 'year', 'month','revisit'])

In [26]:
temp.head()

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX,FLC,year,month,revisit
0,0,0,1,1,1,2020,4,0
1,0,0,1,1,1,2020,4,2
2,0,0,1,1,1,2020,4,1
3,0,0,1,1,1,2020,7,0
4,0,0,1,1,1,2020,7,2


In [27]:
temp=temp[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX', 'FLC','revisit','year', 'month',]]

In [28]:
# 모델 확인
lgbm_reg

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.8, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=15, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [29]:
lgbm_reg.fit(X_train,y_train)
pred=lgbm_reg.predict(temp)
pred=np.expm1(pred)

temp['AMT']=np.round(pred,0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [30]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,4.816346e+08
1,202004,강원,골프장 운영업,1.902673e+10
2,202004,강원,과실 및 채소 소매업,8.596681e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.224251e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,6.622669e+07
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.351645e+09
1390,202007,충북,한식 음식점업,4.723002e+10
1391,202007,충북,호텔업,4.980101e+07
1392,202007,충북,화장품 및 방향제 소매업,1.121023e+09


In [31]:
submission = pd.read_csv('/content/gdrive/My Drive/Jeju_credit contest/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('/content/gdrive/My Drive/Jeju_credit contest/jeju_data_ver1/final-final-submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,4.816346e+08
1,202004,강원,골프장 운영업,1.902673e+10
2,202004,강원,과실 및 채소 소매업,8.596681e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.224251e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,6.622669e+07
